In [1]:
%pip install nltk
%pip install tweepy
%pip install pandas
%pip install wordcloud
%pip install textblob
%pip install seaborn
%pip install poyraz

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

ERROR: Could not find a version that satisfies the requirement poyraz (from versions: none)
ERROR: No matching distribution found for poyraz


In [2]:
import tweepy
import json
import csv
import sys
import time
import re
import warnings
import string
import pandas as pd
import ast
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
from wordcloud import WordCloud
from textblob import TextBlob
from sklearn.neural_network import NeuralNetwork
from sklearn.svm import SVM
from sklearn.metrics import StandardMetrics
from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cemke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
class getTweets:
    def __init__(self,ckey,csecret,atoken,asecret):
        self.api=None
        self.consumer_key = ckey
        self.consumer_secret = csecret
        self.access_token = atoken
        self.access_token_secret = asecret
        
    def setAuth(self):
        try:
            self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
            self.auth.set_access_token(self.access_token, self.access_token_secret)
            self.api = tweepy.API(self.auth)            
            print ("Bağlantı Yapıldı!")                           
        except tweepy.TweepError as err:
            print('Error: {}'.format(err))   
    
    def toDataFrame(tweets):
        try:
            print ("Veriler Kayıt Ediliyor...")
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_colwidth', None)
            DataSet = pd.DataFrame()

            DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
            DataSet['Tweet'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Degistirilmis'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Ceviri']=[tweet.text for tweet in tweets]
            DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]
            DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
            DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]       
            DataSet['Dil'] = [tweet.lang for tweet in tweets]
            print ("Veriler Kayıt Edildi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
    
    def searchTweet(self,aranan):
        self.setAuth()
        try:                      
            print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
            cursor = tweepy.Cursor(self.api.search_tweets, q=aranan, result_type="new", geocode="55.0000,4.0000,1000km",lang='tr')
            results=[]
            for item in cursor.items(100):
                    results.append(item)
            print ("Kelimeye Uygun Tweetler Sorgulandı!")
            DataSet = getTweets.toDataFrame(results)  
            getTweets.TweetTemizle(DataSet)
            DataSet.to_csv('tw.csv',index=False)
        except Exception as err:
            print('Error: {}'.format(err))
                  
    def Ceviri(DataSet):
        try:
            print ("Tweetler Çeviriliyor...")
            for yazi in range(len(DataSet["Tweet"])):
                Ceviri_Blob=TextBlob(DataSet["Tweet"][yazi])
                Eng_Blob=Ceviri_Blob.translate(from_lang='tr', to='en')  
                DataSet['Tweet_Ceviri'][yazi]=Eng_Blob
            print ("Tweetler Çevirildi!")
            return DataSet        
        except Exception as err:
            print('Error: {}'.format(err))
            
    def TweetTemizle(Dataset):
        try:
            print ("Tweetler Temizleniyor...")
            pattern = re.compile('http[s]?://(?:[A-Za-z0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|#[A-Za-z0-9]+|$[A-Za-z0-9]+|@[A-Za-z0-9_]+|[,.^_$*%-;鶯!?’"“”:=|…]+|\n')
            for i in range(len(Dataset["Tweet_Degistirilmis"])):        
                Dataset["Tweet_Degistirilmis"][i]=pattern.sub(' ', Dataset["Tweet_Degistirilmis"][i])
            print ("Tweetler Temizlendi!")
            getTweets.Ceviri(Dataset)
            return Dataset
        except Exception as err:
            print('Error: {}'.format(err))
    
    def KelimeAyristirma(DataSet):
        try:
            data_test = DataSet.copy()
            stop = stopwords.words('english')
            data_test["Tweet_Degistirilmis"] = data_test["Tweet_Degistirilmis"].str.lower().str.split()
            data_test["Tweet_Degistirilmis"] = data_test["Tweet_Degistirilmis"].apply(lambda x: [item for item in x if item not in stop])
        except Exception as err:
            print('Error: {}'.format(err))
    
    def turkceAnaliz(self,Dataset):
        return 
    
    def getSubjectivity(self,Dataset):
        return TextBlob(Dataset).sentiment.subjectivity
    
    def getPolarity(self,Dataset):
        return TextBlob(Dataset).sentiment.polarity 


In [4]:
tw = getTweets( 
ckey = 'o7qidRODRWm5u07PtAWrCpSqf',
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV',
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc',
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'
)
aranan=input("Aranacak Kelimeyi Girin: ")
tw.searchTweet(aranan)

Bağlantı Yapıldı!
Kelimeye Uygun Tweetler Sorgulanıyor...
Kelimeye Uygun Tweetler Sorgulandı!
Veriler Kayıt Ediliyor...
Veriler Kayıt Edildi!
Tweetler Temizleniyor...
Tweetler Temizlendi!
Tweetler Çeviriliyor...
Tweetler Çevirildi!


In [5]:
Veri=pd.read_csv('tw.csv')
Veri.head()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil
0,1604591849532186624,@erkanabe dünya kupası da bittiğine göre bitkisel hayattan çıkalım şöyle https://t.co/XyDIavMAcx,dünya kupası da bittiğine göre bitkisel hayattan çıkalım şöyle,"@Erkanabe World Cup is also finished, let's get out of herbal life as follows https://t.co/xydiavmacx",2022-12-18 21:38:03+00:00,1229262853934284800,gherok,tr
1,1604590341138649089,@fatihaltayli Gördügüm ve yasadigim en harika Dünya Kupasi idi.Bati ve onun TR deki kuklalari tabiki Camur atma yar… https://t.co/8VXFjpfP9h,Gördügüm ve yasadigim en harika Dünya Kupasi idi Bati ve onun TR deki kuklalari tabiki Camur atma yar,@fatihaltayli was the most wonderful world cup that I saw and lived.,2022-12-18 21:32:03+00:00,497221016,göcmen musa,tr
2,1604590214479060997,2026 Dünya Kupası’nın en büyük şampiyonluk adayı Fransa’dır.,Dünya Kupası nın en büyük şampiyonluk adayı Fransa dır,The largest championship candidate for the 2026 World Cup is France.,2022-12-18 21:31:33+00:00,44848789,auysal4,tr
3,1604590147756261376,@AyhanArslann1 @EnSuperLig Renktaş 1 zaten sen paranı veriyorsdun vergi olarak trt ye bedava izlemiyorsun yani :) 2… https://t.co/zhGlgBjFmF,Renktaş zaten sen paranı veriyorsdun vergi olarak trt ye bedava izlemiyorsun yani,"@Ayhanarslann1 @ensuperlig colortaş 1 you already give your money, you don't watch it free of charge as a tax :) 2… https://t.co/zhglgbjfmf",2022-12-18 21:31:17+00:00,1562534300482158594,Berk CanDemiR ❤️💛💪,tr
4,1604590118492598272,"@haciykk Sizde adam gibi göçmen ithal etseydiniz de bari dünya kupası vs bişeyler yapsaydık. Afgan, Pakistanlı, Sur… https://t.co/W8fir6fKD8",Sizde adam gibi göçmen ithal etseydiniz de bari dünya kupası vs bişeyler yapsaydık Afgan Pakistanlı Sur,"@haciykk If you imported immigrants like a man, we would have done something in the world cup etc. Afghan, Pakistanli, Sur… https://t.co/w8fir6fkd8",2022-12-18 21:31:10+00:00,1464609803137859592,Greg,tr


In [6]:
Veri.tail()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil
95,1604552929930780673,Şu katarlılar amk herkesi para ile aldılar messi yıllarca beklediği anda ne hala sokmuşlar bu dünya kupasını izleme… https://t.co/100GkA3NPu,Şu katarlılar amk herkesi para ile aldılar messi yıllarca beklediği anda ne hala sokmuşlar bu dünya kupasını izleme,"Those Qatarlılar amk took everyone with money, Messi, what they still stuck in the moment they waited for years, don't watch this world cup… https://t.co/100gka3npu",2022-12-18 19:03:24+00:00,947181197431623680,Mustafa Mehmet Coratekin,tr
96,1604552925207986176,2022 dünya kupası şu an ile bile hatırlanabilir. Yürek koydu resmen! https://t.co/Xotpexd7TO,dünya kupası şu an ile bile hatırlanabilir Yürek koydu resmen,The 2022 World Cup can be remembered even now. He put the heart officially! https://t.co/xotpexd7to,2022-12-18 19:03:23+00:00,320300737,breakingdad,tr
97,1604552826356699136,@VeliIsmail1 Ben 1990dan beri izliyorum. Benim için en berbat dünya kupası Rusya'daki ile birlikte ama evet en iyi… https://t.co/LeKWzi160l,Ben dan beri izliyorum Benim için en berbat dünya kupası Rusya daki ile birlikte ama evet en iyi,@Veliısmail1 I have been watching since 1990. The worst world cup for me is with Russia but yes the best… https://t.co/lekwzi160L,2022-12-18 19:02:59+00:00,1487719412673257477,Alperinho79,tr
98,1604552791762296832,"Dünya Kupası heyecanının maddi bir değeri olsaydı(zaten var da, daha somut olarak) kaç TL öderdiniz?",Dünya Kupası heyecanının maddi bir değeri olsaydı zaten var da daha somut olarak kaç TL öderdiniz,"If the excitement of the world cup had a financial value (there is already, how much would you pay for more than concrete)?",2022-12-18 19:02:51+00:00,2766680320,Tamarisk,tr
99,1604552739098353664,Kurgulanmış bir dünya kupası,Kurgulanmış bir dünya kupası,A fictional world cup,2022-12-18 19:02:38+00:00,1666182349,NilaySneijderoğlu 🐺,tr


In [7]:
savedTweets = Veri
savedTweets['Subjectivity'] = savedTweets['Tweet_Ceviri'].apply(tw.getSubjectivity)
savedTweets['Polarity'] = savedTweets['Tweet_Ceviri'].apply(tw.getPolarity)
savedTweets['Duygu Turkce'] = savedTweets['Tweet_Ceviri'].apply(tw.turkceAnaliz)
savedTweets.to_csv('duygu.csv')
savedTweets.head()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Subjectivity,Polarity,Duygu Turkce
0,1604591849532186624,@erkanabe dünya kupası da bittiğine göre bitkisel hayattan çıkalım şöyle https://t.co/XyDIavMAcx,dünya kupası da bittiğine göre bitkisel hayattan çıkalım şöyle,"@Erkanabe World Cup is also finished, let's get out of herbal life as follows https://t.co/xydiavmacx",2022-12-18 21:38:03+00:00,1229262853934284800,gherok,tr,0.00,0.00,None
1,1604590341138649089,@fatihaltayli Gördügüm ve yasadigim en harika Dünya Kupasi idi.Bati ve onun TR deki kuklalari tabiki Camur atma yar… https://t.co/8VXFjpfP9h,Gördügüm ve yasadigim en harika Dünya Kupasi idi Bati ve onun TR deki kuklalari tabiki Camur atma yar,@fatihaltayli was the most wonderful world cup that I saw and lived.,2022-12-18 21:32:03+00:00,497221016,göcmen musa,tr,0.75,0.75,None
2,1604590214479060997,2026 Dünya Kupası’nın en büyük şampiyonluk adayı Fransa’dır.,Dünya Kupası nın en büyük şampiyonluk adayı Fransa dır,The largest championship candidate for the 2026 World Cup is France.,2022-12-18 21:31:33+00:00,44848789,auysal4,tr,0.00,0.00,None
3,1604590147756261376,@AyhanArslann1 @EnSuperLig Renktaş 1 zaten sen paranı veriyorsdun vergi olarak trt ye bedava izlemiyorsun yani :) 2… https://t.co/zhGlgBjFmF,Renktaş zaten sen paranı veriyorsdun vergi olarak trt ye bedava izlemiyorsun yani,"@Ayhanarslann1 @ensuperlig colortaş 1 you already give your money, you don't watch it free of charge as a tax :) 2… https://t.co/zhglgbjfmf",2022-12-18 21:31:17+00:00,1562534300482158594,Berk CanDemiR ❤️💛💪,tr,0.90,0.45,None
4,1604590118492598272,"@haciykk Sizde adam gibi göçmen ithal etseydiniz de bari dünya kupası vs bişeyler yapsaydık. Afgan, Pakistanlı, Sur… https://t.co/W8fir6fKD8",Sizde adam gibi göçmen ithal etseydiniz de bari dünya kupası vs bişeyler yapsaydık Afgan Pakistanlı Sur,"@haciykk If you imported immigrants like a man, we would have done something in the world cup etc. Afghan, Pakistanli, Sur… https://t.co/w8fir6fkd8",2022-12-18 21:31:10+00:00,1464609803137859592,Greg,tr,0.00,0.00,None


In [8]:
savedTweets.tail()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Subjectivity,Polarity,Duygu Turkce
95,1604552929930780673,Şu katarlılar amk herkesi para ile aldılar messi yıllarca beklediği anda ne hala sokmuşlar bu dünya kupasını izleme… https://t.co/100GkA3NPu,Şu katarlılar amk herkesi para ile aldılar messi yıllarca beklediği anda ne hala sokmuşlar bu dünya kupasını izleme,"Those Qatarlılar amk took everyone with money, Messi, what they still stuck in the moment they waited for years, don't watch this world cup… https://t.co/100gka3npu",2022-12-18 19:03:24+00:00,947181197431623680,Mustafa Mehmet Coratekin,tr,0.00,0.0000,None
96,1604552925207986176,2022 dünya kupası şu an ile bile hatırlanabilir. Yürek koydu resmen! https://t.co/Xotpexd7TO,dünya kupası şu an ile bile hatırlanabilir Yürek koydu resmen,The 2022 World Cup can be remembered even now. He put the heart officially! https://t.co/xotpexd7to,2022-12-18 19:03:23+00:00,320300737,breakingdad,tr,0.00,0.0000,None
97,1604552826356699136,@VeliIsmail1 Ben 1990dan beri izliyorum. Benim için en berbat dünya kupası Rusya'daki ile birlikte ama evet en iyi… https://t.co/LeKWzi160l,Ben dan beri izliyorum Benim için en berbat dünya kupası Rusya daki ile birlikte ama evet en iyi,@Veliısmail1 I have been watching since 1990. The worst world cup for me is with Russia but yes the best… https://t.co/lekwzi160L,2022-12-18 19:02:59+00:00,1487719412673257477,Alperinho79,tr,1.00,-1.0000,None
98,1604552791762296832,"Dünya Kupası heyecanının maddi bir değeri olsaydı(zaten var da, daha somut olarak) kaç TL öderdiniz?",Dünya Kupası heyecanının maddi bir değeri olsaydı zaten var da daha somut olarak kaç TL öderdiniz,"If the excitement of the world cup had a financial value (there is already, how much would you pay for more than concrete)?",2022-12-18 19:02:51+00:00,2766680320,Tamarisk,tr,0.25,0.2125,None
99,1604552739098353664,Kurgulanmış bir dünya kupası,Kurgulanmış bir dünya kupası,A fictional world cup,2022-12-18 19:02:38+00:00,1666182349,NilaySneijderoğlu 🐺,tr,0.25,0.0000,None
